<a href="https://colab.research.google.com/github/luizaleao/PTR/blob/main/SemiDroid_testes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementação artigo SemiDroid**


In [112]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import random
import math
import timeit
from sklearn.svm import SVC
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KDTree
from collections import Counter
from math import log
from scipy import stats
from scipy.stats import norm
from scipy.stats import entropy

# Dataset

In [113]:
dataset = pd.read_csv('md46k_all.csv')
dataset

,Permission :: WAKE_LOCK,Permission :: WRITE_EXTERNAL_STORAGE,Permission :: ACCESS_NETWORK_STATE,Permission :: WRITE_SETTINGS,Permission :: INTERNET,Permission :: ACCESS_WIFI_STATE,Permission :: ACCESS_FINE_LOCATION,Permission :: READ_GSERVICES,Permission :: ACCESS_COARSE_LOCATION,Permission :: GET_ACCOUNTS,...,API Call :: Landroid/renderscript/RenderScript.setMessageHandler(),API Call :: Landroid/widget/PullingLayout.o(),API Call :: Landroid/content/ContextWrapper.getExternalMediaDirs(),API Call :: Landroid/widget/ExpandableListView.onDraw(),API Call :: Landroid/icu/util/TimeZone.getID(),API Call :: Landroid/os/Parcel.writeTypedObject(),API Call :: Landroid/text/TextPaint.setStrokeCap(),API Call :: Landroid/telecom/Connection.createFailedConnection(),API Call :: Landroid/widget/CheckBox.getGlobalVisibleRect(),class
0,0,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5681,0,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5682,1,1,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1
5683,1,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5684,1,1,1,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [114]:
from pathlib import Path

In [115]:
file_path = 'md46k_all.csv'
file_name = Path(file_path).stem
dataset_name = file_name

In [116]:
dataset_name


'md46k_all'

# **Funções Auxiliares**

In [117]:
def distance(x1, x2):
    return sum((x1-x2)**2)

In [118]:
def FilterFeatures(ft_names, coef):
  i=0
  n1 = len(ft_names)

  features_list_to_delete = []

  for i in range(n1):
     if coef[i] < 1.0:
       features_list_to_delete.append(ft_names[i])

  return (features_list_to_delete)

In [119]:
def _Ex_a_v_(Ex, a, v, nan=True): 
  if nan:
      return [x for x, t in zip(Ex, a) if (isinstance(t, float) and
                                            isinstance(v, float) and
                                            math.isnan(t)        and
                                            math.isnan(v))       or
                                            (t == v)]
  else:
      return [x for x, t in zip(Ex, a) if t == v]


def intrinsic_value(Ex, a, nan=True):
  sum_v = 0
  for v in set(a):
    Ex_a_v = _Ex_a_v_(Ex, a, v, nan)  
    if len(Ex_a_v) != 0: 
      return  (len(Ex_a_v) / len(Ex))* (log(len(Ex_a_v) / len(Ex), 2))


  

# **Funções de Ranking**

In [120]:
#Função ChiSquared
def chisquared(X,y):
  feature_names = X.columns
  chi2score, pval = chi2(X, y)
  data = {"features": feature_names, "score": chi2score}
  data1 = pd.DataFrame(data)
  data1 = data1.sort_values(by=['score'], ascending=False)
  #n = len(feature_names)
  #indice = int(log(n,2))
  indice = 30
  features_to_delete = data1.features[indice:]
  features_to_delete = pd.DataFrame(features_to_delete).T
  features_to_delete = features_to_delete.values.tolist()

  new_X = X.drop(columns=features_to_delete[0])
  new_X['class'] = y
  df = pd.DataFrame(new_X)
  return df 

#Função Information Gain
def i_gain(Ex, a, nan=True):
  H_Ex = entropy(list(Counter(Ex).values()))
  sum_v = 0
  for v in set(a):
      Ex_a_v = _Ex_a_v_(Ex, a, v, nan)

      sum_v += (len(Ex_a_v) / len(Ex)) *\
                (entropy(list(Counter(Ex_a_v).values())))

  result = H_Ex - sum_v
  return result

def info_gain(X,y):
  feature_names = X.columns
  ig_df = pd.DataFrame(columns=["features", "score"])
  for i in dataset.columns:
      if i != "class":
          ig_df = pd.concat([ig_df, pd.DataFrame([[i, i_gain(i,dataset)]], columns=["features", "score"])])
  data1 = ig_df.sort_values(by="score", ascending=False)
  #n = len(feature_names)
  #indice = int(log(n,2))
  indice = 30
  features_to_delete = data1.features[indice:]
  features_to_delete = pd.DataFrame(features_to_delete).T
  features_to_delete = features_to_delete.values.tolist()

  new_X = X.drop(columns=features_to_delete[0])
  new_X['class'] = y
  df = pd.DataFrame(new_X)
  return df 

#Função Gain Ratio
def g_ratio(Ex, a, nan=True):
  result = i_gain(Ex, a) /(- intrinsic_value(Ex, a) )
  return result
    
def gain_ratio(X,y):
  feature_names = X.columns
  ig_df = pd.DataFrame(columns=["features", "score"])
  for i in dataset.columns:
      if i != "class":
          ig_df = pd.concat([ig_df, pd.DataFrame([[i, g_ratio(i,dataset)]], columns=["features", "score"])])
  data1 = ig_df.sort_values(by="score", ascending=False)
  #n = len(feature_names)
  #indice = int(log(n,2))
  indice = 30
  features_to_delete = data1.features[indice:]
  features_to_delete = pd.DataFrame(features_to_delete).T
  features_to_delete = features_to_delete.values.tolist()

  new_X = X.drop(columns=features_to_delete[0])
  new_X['class'] = y
  df = pd.DataFrame(new_X)
  return df

#Função OneR
def OneR(X, y):
  feature_names = X.columns
  ideal_variable = 0
  max_accuracy = 0
  
  response = list()
  result = dict()
  
  dfx = pd.DataFrame(X)
  
  for i in dfx:
      result[str(i)] = dict()
      options_values = set(dfx[i])
      join_data = pd.DataFrame({"variable":dfx[i], "label":y})
      cross_table = pd.crosstab(join_data.variable, join_data.label)
      summary = cross_table.idxmax(axis=1)
      result[str(i)] = dict(summary)
      
      counts = 0
      
      for idx, row in join_data.iterrows():
          if row['label'] == result[str(i)][row['variable']]:
              counts += 1

      accuracy = (counts/len(y))
      
      if accuracy > max_accuracy:
          max_accuracy = accuracy
          ideal_variable = i
      result_feature = {"features": str(i), "score":accuracy }  
      response.append(result_feature)

  data1 = pd.DataFrame(response)
  data1 = data1.sort_values(by=['score'], ascending=False)
  #n = len(feature_names)
  #indice = int(log(n,2))
  indice = 30
  features_to_delete = data1.features[indice:]
  features_to_delete = pd.DataFrame(features_to_delete).T
  features_to_delete = features_to_delete.values.tolist()

  new_X = X.drop(columns=features_to_delete[0])
  new_X['class'] = y
  df = pd.DataFrame(new_X)
  return df
    
#Função Principal Component Analysis 
def pcanalysis(X,y):
  values = X.values
  x = StandardScaler().fit_transform(values)
  principal=PCA(n_components=1)
  principalComponents = principal.fit_transform(x)
  features_names = np.array(X.columns.values.tolist())
  fold_count = [10] * len(features_names)
  fold_ft_num = []
  fold_ft_to_delete = []

  to_drop = set()

  ft_to_delete = FilterFeatures(features_names, principalComponents)

  fold_ft_num.append(len(ft_to_delete))
  fold_ft_to_delete.append(ft_to_delete)

  for ft in ft_to_delete:
      index = list(features_names).index(ft)
      fold_count[index]-=1
      to_drop.add(ft)
      
  new_X = X.drop(columns = to_drop)
  new_X['class'] = y
  df = pd.DataFrame(new_X)
  return df 
 
#Função Logistic Regression
def logistic_regression(X,y):
  feature_names = X.columns
  model = LogisticRegression().fit(X,y)
  a = pd.DataFrame(feature_names)

  score = model.coef_
  score = pd.DataFrame(score).T

  data1 = pd.concat([a,score],1)
  data1.columns = ['features', 'score']
  data1 = data1.sort_values(by=['score'], ascending=False)
  a = data1['score']
  b = data1['features']
  lista1 = a.values.tolist()
  lista2 = b.values.tolist()
  c = len(lista1)
  i=0
  n1 = len(feature_names)

  features_to_delete = []

  for i in range(c):
    if a[i] < 0.0:
      features_to_delete.append(b[i])

  features_to_delete = pd.DataFrame(features_to_delete).T
  features_to_delete = features_to_delete.values.tolist()

  new_X = X.drop(columns=features_to_delete[0])
  new_X['class'] = y
  df = pd.DataFrame(new_X)
  return df 

# Min-Max Normalization

In [121]:
def minmax(df):
  df_min_max_scaled = df.copy() 
  for column in df_min_max_scaled.columns: 
      df_min_max_scaled[column] =(df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())
  return df_min_max_scaled


# Algoritmos ML

In [122]:
def SVM(dataset):
    from sklearn import metrics
    state = np.random.randint(100)
    X = dataset.drop(columns = ['class']) #variaveis (features)
    Y = dataset['class'] 

    start_time = timeit.default_timer()
    #split between train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.3,random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

    svm = SVC(kernel='linear', C=1.0, random_state=1)
    svm.fit(X_train,y_train)

    #prediction labels for X_test
    y_pred=svm.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    elapsed_time = timeit.default_timer() - start_time

    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, zero_division = 0)
    recall = metrics.recall_score(y_test, y_pred, zero_division = 0)
    f1_score = metrics.f1_score(y_test, y_pred, zero_division = 0)
    fpr = fp/(fp+tn)

    data = [{'Time':elapsed_time,'Accuracy':accuracy,'Precision':precision,'Recall':recall,'F1_Score':f1_score,'Fpr':fpr}]
    df = pd.DataFrame(data)
    return df


In [123]:
def RF(dataset):
    from sklearn import metrics
    state = np.random.randint(100)
    X = dataset.drop(columns = ['class']) #variaveis (features)
    Y = dataset['class'] 

    start_time = timeit.default_timer()
    #split between train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.3,random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

    RF = RandomForestClassifier()
    RF.fit(X_train,y_train)

    #prediction labels for X_test
    y_pred=RF.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    elapsed_time = timeit.default_timer() - start_time

    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, zero_division = 0)
    recall = metrics.recall_score(y_test, y_pred, zero_division = 0)
    f1_score = metrics.f1_score(y_test, y_pred, zero_division = 0)
    fpr = fp/(fp+tn)

    data = [{'Time':elapsed_time,'Accuracy':accuracy,'Precision':precision,'Recall':recall,'F1_Score':f1_score,'Fpr':fpr}]
    df = pd.DataFrame(data)
    return df

In [124]:
def data_selection(X,y):
  FS1 = chisquared(X,y)
  FS2 = info_gain(X,y)
  FS3 = gain_ratio(X,y)
  FS4 = OneR(X,y)
  FS5 = pcanalysis(X,y)
  FS6 = logistic_regression(X,y)
  fs1 = FS1.to_csv(dataset_name+"_chisquared.csv")
  fs2 = FS2.to_csv(dataset_name+"_infogain.csv")
  fs3 = FS3.to_csv(dataset_name+"_gainratio.csv")
  fs4 = FS4.to_csv(dataset_name+"_oner.csv")
  fs5 = FS5.to_csv(dataset_name+"_pca.csv")
  fs6 = FS6.to_csv(dataset_name+"_logisticregression.csv")
  return FS1, FS2, FS3, FS4, FS5, FS6

In [125]:
def data_min_max(X,y):
  FS1, FS2, FS3, FS4, FS5, FS6 = data_selection(X,y)

  data1 = minmax(FS1)
  data2 = minmax(FS2)
  data3 = minmax(FS3)
  data4 = minmax(FS4)
  data5 = minmax(FS5)
  data6 = minmax(FS6)

  return data1, data2, data3, data4, data5, data6

In [126]:
def rf_train(X,y):
  data1, data2, data3, data4, data5, data6 = data_min_max(X,y)
  name_csv = ["chisquared.csv","infogain.csv","gainratio.csv","oner.csv","pca.csv","logisticregression.csv"]
  ML1 = RF(data1)
  ML2 = RF(data2)
  ML3 = RF(data3)
  ML4 = RF(data4)
  ML5 = RF(data5)
  ML6 = RF(data6)
  ml = [ML1,ML2,ML3,ML4,ML5,ML6]
  for i in range(len(name_csv)):
    ml[i].to_csv("RF_"+dataset_name+"_"+name_csv[i])
  return ML1, ML2, ML3, ML4, ML5, ML6

In [127]:
def svm_train(X,y):
  data1, data2, data3, data4, data5, data6 = data_min_max(X,y)
  name_csv = ["chisquared.csv","infogain.csv","gainratio.csv","oner.csv","pca.csv","logisticregression.csv"]
  ML1 = SVM(data1)
  ML2 = SVM (data2)
  ML3 = SVM(data3)
  ML4 = SVM(data4)
  ML5 = SVM(data5)
  ML6 = SVM(data6)
  ml = [ML1,ML2,ML3,ML4,ML5,ML6]
  for i in range(len(name_csv)):
    ml[i].to_csv("SVM_"+dataset_name+"_"+name_csv[i])
  return ML1, ML2, ML3, ML4, ML5, ML6

In [ ]:
if __name__ == '__main__':
  
  X = dataset.drop(columns = ['class']) #variaveis (features)
  y = dataset['class'] 
  
  rf_train(X,y)
  svm_train(X,y)

In [ ]:
dataset = pd.read_csv('md46k_all_chisquared.csv')
dataset

In [ ]:
dataset = pd.read_csv('md46k_all_infogain.csv')
dataset

In [ ]:
dataset = pd.read_csv('md46k_all_gainratio.csv')
dataset

In [ ]:
dataset = pd.read_csv('drebin_215_all_oner.csv')
dataset

In [ ]:
dataset = pd.read_csv('md46k_all_pca.csv')
dataset

In [ ]:
dataset = pd.read_csv('md46k_all_logisticregression.csv')
dataset